<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

# [Tutoriel d'apprentissage automatique](https://github.com/wikistat/MLTraining)

# *NLP* et Catégorisation de Produits en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="R"/></a> avec <a href="http://scikit-learn.org/stable/#"><img src="http://scikit-learn.org/stable/_static/scikit-learn-logo-small.png" style="max-width: 100px; display: inline" alt="scikit-learn"/></a> et <a href="https://radimrehurek.com/gensim/index.html" ><img src="https://radimrehurek.com/gensim/_static/images/gensim.png" style="max-width: 100px; display: inline" alt="gensim"/></a> 

#### Résumé
Le principal objectif est d'illuster sur des vraies données les processus d'analyse de données textuelles ou de langage naturel (NLP). Plusieurs étapes sont considérées et exécutées dans l'environnement Pyhton avec le slibrairies spécialisées. Nesttoyage des textes, éliminaiton des mots inutiles, racinisation ainsi que la comparaison de différentes stratgies de vectorisation (comptage, TF-IDF, word2vect). A la suite de quoi l'onjectif de classificaiton en catégorie est atteint en comparant deux méthodes: régresison logistique et forêts aléatoires.


## 1 Introduction

Il s'agit d'une version simplifiée du concours proposé par CDiscount et paru sur le site [datascience.net](https://www.datascience.net/fr/challenge) puis Kaggle. Les données d'apprentissage sont accessibles sur demande auprès de CDiscount. Les solutions de l'échantillon test du concours ne sont pas et ne seront pas rendues publiques. Un échantillon test est donc construit pour l'usage de ce tutoriel.  L'objectif est de prévoir la catégorie d'un produit à partir de son descriptif. Seule la catégorie principale (1er niveau) est prédite au lieu des trois niveaux demandés dans le concours. L'objectif est d'illustrer sur un exemple complexe le prétraitement de données textuelles ou de langave naturel (*Natural Language Process - NLP*). La stratégie de sous ou sur échantillonnage des catégories qui permet d'améliorer la prévision n'a pas été mise en oeuvre.
* L'exemple est présenté sur un échantillon réduit d'un million de produits au lieu des 15M initiaux
* L'échantillon réduit peut encore l'être puis séparé en 2 parties: apprentissage et validation. 
* Les données textuelles sont  nettoyées, racinisées, vectorisées avant modélisation.
* Deux modélisations sont estimées: logistique, forêts aléatoires.

# <FONT COLOR="Red">Épisode 1</font>

## 2 Prise en charge des données
### 2.1 Environnement

In [ ]:
#Importation des librairies utilisées
import unicodedata 
import time
import pandas as pd
import numpy as np
import random
import nltk
import collections
import itertools
import warnings

import matplotlib.pyplot as plt
import seaborn as sb
sb.set_style("whitegrid")

import sklearn.model_selection as scv

**Attetnion** Si vous utilisez la librairie `nltk` pour la première fois dans votre environnement, il est nécessaire d'exécuter la commande suivante. Cette commande permet de télécharger de nombreux corpus de texte, mais également des informations grammaticales sur différentes langues; information notamment nécessaire à l'étape de racinisation.

In [ ]:
# nltk.download()

### 2.2 Lecture des données

* `cdiscount_train`: 1.000.000 de lignes
* `cdisount_test`: 50.000 lignes

Définition d'une fonction permettant de lire le fichier d'apprentissage et de créer aléatoirement deux *data frames  `Pandas`, un pour l'apprentissage, l'autre pour le test. 

In [ ]:
def split_dataset(input_path, nb_line, tauxTest):
    data_all = pd.read_csv(input_path,sep=",", nrows=nb_line)
    data_all = data_all.fillna("")
    data_train, data_test = scv.train_test_split(data_all, test_size = tauxTest)
    time_end = time.time()
    return data_train, data_test

La taille du fichier est limité à `nb_line=10 000` pour réduire les temps de calcul à venir. Ce nombre peut être augmenté afin de tester une amélioraiton des performances mais au prix du temps de calcul. Cette analyse systématique de l'impact de la taille de l'échantillon est développé par [Besse et al. (2017)](https://hal.archives-ouvertes.fr/hal-01350099v3) qui opèrent de plus une comparaison des performances entre R, Python et Spark. 

In [ ]:
input_path = "data/cdiscount_train.csv.zip"
nb_line=100000  # part totale extraite du fichier initial ici déjà réduit
tauxTest = 0.05 # proportion de l'échantillon test
data_train, data_test = split_dataset(input_path, nb_line, tauxTest)
# Cette ligne permet de visualiser les 5 premières lignes du DataFrame 
N_train = data_train.shape[0]
N_test = data_test.shape[0]
print("Train set : %d elements, Test set : %d elements" %(N_train, N_test))
data_train.head(5)

### 2.3 Exploration élémentaire

In [ ]:
# Liste des catégories du premier niveau
data_train.groupby("Categorie1").first()[["Description","Marque"]]

Dénombrement des catégories du premier niveau.

In [ ]:
#Count occurence of each Categorie
data_count = data_train["Categorie1"].value_counts()
#Rename index to add percentage
new_index = [k+ ": %.2f%%" %(v*100/N_train) for k,v in data_count.iteritems()]
data_count.index= new_index

fig=plt.figure(figsize= (10,10))
ax = fig.add_subplot(1,1,1)
data_count.plot.barh(logx = False)
plt.show()

Que peut-on dire sur la distribution des ce classes?

Les données sont enregistrées dans les fichiers `train` et `test` de type `csv` pour des réutilisations ultérieures.

In [ ]:
data_test.to_csv("data/cdiscount_test.csv", index=False)
data_train.to_csv("data/cdiscount_train_subset.csv", index=False)

##  3. Nettoyage des données
Afin de limiter la dimension de l'espace des variables ou *features*, tout en conservant les informations essentielles, il est nécessaire de nettoyer les données en appliquant plusieurs étapes:
* Chaque mot est écrit en minuscule.
* Les termes numériques, de ponctuation et autres symboles sont supprimés.
* 155 mots-courants, et donc non informatifs, de la langue française sont supprimés (STOPWORDS). Ex: le, la, du, alors, etc...
* Chaque mot est "racinisé", via la fonction `STEMMER.stem` de la librairie nltk. La racinisation transforme un mot en son radical ou sa racine. Par exemple, les mots: cheval, chevaux, chevalier, chevalerie, chevaucher sont tous remplacés par "cheva".

### 3.1 Importation des librairies et fichier pour le nettoyage des données.

In [ ]:
# Librairies 
from bs4 import BeautifulSoup #Nettoyage d'HTML
import re # Regex
import nltk # bibliothèque de fonctions NLP

## listes de mots à supprimer dans la description des produits
## Depuis NLTK
nltk_stopwords = nltk.corpus.stopwords.words('french') 
## Depuis Un fichier externe.
lucene_stopwords =open("data/lucene_stopwords.txt","r").read().split(",") #En local
## Union des deux fichiers de stopwords 
stopwords = list(set(nltk_stopwords).union(set(lucene_stopwords)))

## Fonction de stemming ou de racinisation
stemmer=nltk.stem.SnowballStemmer('french')

### 3.2 Illustration sur un descriptif 

**Ligne Originale**

In [ ]:
i = 0
description = data_train.Description.values[i]
print("Original Description : " + description)

** Supprimer les posibles balises HTML dans la description **

In [ ]:
# txt = BeautifulSoup(description,"html.parser",from_encoding='utf-8').get_text()
txt = BeautifulSoup(description,"html.parser").get_text()
print(txt)

** Convertir le texte en minuscule **

In [ ]:
txt = txt.lower()
print(txt)

** Remplacer quelques caractères spéciaux **

* `\u2026`: `…`
* `\u00a0`: `NO-BREAK SPACE`

Cette liste peut être compléter en fonction du jeu de donées étudiés

In [ ]:
txt = txt.replace(u'\u2026','.')    
txt = txt.replace(u'\u00a0',' ')
print(txt)

** Supprimer les accents **

In [ ]:
txt = unicodedata.normalize('NFD', txt).encode('ascii', 'ignore').decode("utf-8")
print(txt)

** Supprimer les caractères qui ne sont ne sont pas des lettres minuscules **

In [ ]:
txt = re.sub('[^a-z_]', ' ', txt)
print(txt)

** Remplacer la description par une liste de mots (tokens), supprimer les mots de moins de 2 lettres ainsi que les stopwords **


In [ ]:
tokens = [w for w in txt.split() if (len(w)>2) and (w not in stopwords)]
print(tokens)

** Raciniser (stem) chaque tokens **

In [ ]:
tokens_stem = [stemmer.stem(token) for token in tokens]
print(tokens_stem)

### 3.3 Fonctions de nettoyage de texte
Cette fonction qui prend en entrée un texte et retourne le texte nettoyé en appliquant successivement les étapes précédentes. 

In [ ]:
# Fonction clean générale
def clean_txt(txt):
    ### remove html stuff
    txt = BeautifulSoup(txt,"html.parser",from_encoding='utf-8').get_text()
    ### lower case
    txt = txt.lower()
    ### special escaping character '...'
    txt = txt.replace(u'\u2026','.')
    txt = txt.replace(u'\u00a0',' ')
    ### remove accent btw
    txt = unicodedata.normalize('NFD', txt).encode('ascii', 'ignore').decode("utf-8")
    ###txt = unidecode(txt)
    ### remove non alphanumeric char
    txt = re.sub('[^a-z_]', ' ', txt)
    ### remove french stop words
    tokens = [w for w in txt.split() if (len(w)>2) and (w not in stopwords)]
    ### french stemming
    tokens_stem = [stemmer.stem(token) for token in tokens]
    ### tokens = stemmer.stemWords(tokens)
    return ' '.join(tokens), " ".join(tokens_stem)

def clean_marque(txt):
    txt = re.sub('[^a-zA-Z0-9]', '_', txt).lower()
    return txt

In [ ]:
from multiprocessing import Pool
p = Pool(4)
    
# fonction de nettoyage du fichier(stemming et liste de mots à supprimer)
def clean_df(input_data, column_names= ['Description', 'Libelle', 'Marque']):
    nb_line = input_data.shape[0]
    print("Start Clean %d lines" %nb_line)
    # Cleaning start for each columns
    time_start = time.time()
    clean_list=[]
    clean_stem_list=[]
    for column_name in column_names:
        column = input_data[column_name].values
        if column_name == "Marque":
            array_clean = np.array(list(p.map(clean_marque,column)))
            clean_list.append(array_clean)
            clean_stem_list.append(array_clean)
        else:
            A = np.array(list(p.map(clean_txt,column)))
            array_clean = A[:,0]
            array_clean_stem = A[:,1]
            clean_list.append(array_clean)
            clean_stem_list.append(array_clean_stem)
    time_end = time.time()
    print("Cleaning time: %d secondes"%(time_end-time_start))
    
    #Convert list to DataFrame
    array_clean = np.array(clean_list).T
    data_clean = pd.DataFrame(array_clean, columns = column_names)
    
    array_clean_stem = np.array(clean_stem_list).T
    data_clean_stem = pd.DataFrame(array_clean_stem, columns = column_names)
    return data_clean, data_clean_stem

### 3.4 Nettoyage des *data frames*
Applique le nettoyage sur toutes les lignes du *data frame* et créé deux nouveaux *data frames*: avec et sans l'étape de racinisation)

In [ ]:
# Take approximately 2 minutes fors 100.000 rows
data_test_clean, data_test_clean_stem = clean_df(data_test)

In [ ]:
data_train_clean, data_train_clean_stem = clean_df(data_train)

Affiche les 5 premières lignes du DataFrame d'apprentissage après nettoyage.

In [ ]:
data_train_clean.head(5)

In [ ]:
data_train_clean_stem.head(5)

Taille du dictionnaire de mots pour le dataset avant et après la racinisation.

In [ ]:
concatenate_text = " ".join(data_train_clean["Description"].values)
list_of_word = concatenate_text.split(" ")
N = len(set(list_of_word))
print(N)

In [ ]:
concatenate_text = " ".join(data_train_clean_stem["Description"].values)
list_of_word_stem = concatenate_text.split(" ")
N = len(set(list_of_word_stem))
print(N)

### 3.4 Représentations par *wordcloud*

In [ ]:
from wordcloud import WordCloud

In [ ]:
all_descr = " ".join(data_test.Description.values)
wordcloud_word = WordCloud(background_color="black").generate_from_text(all_descr)

plt.figure()
plt.imshow(wordcloud_word,cmap=plt.cm.Paired)
plt.axis("off")
plt.show()

In [ ]:
all_descr_clean_stem = " ".join(data_test_clean_stem.Description.values)
wordcloud_word = WordCloud(background_color="black").generate_from_text(all_descr_clean_stem)

plt.figure()
plt.imshow(wordcloud_word,cmap=plt.cm.Paired)
plt.axis("off")
plt.show()

Sauvegarder les jeux de données nettoyés avec et sans racinisation dans des fichiers csv. Cela permet de reprendre l'analyse à la suite d'une interuption.

In [ ]:
data_test_clean.to_csv("data/cdiscount_test_clean.csv", index=False)
data_train_clean.to_csv("data/cdiscount_train_clean.csv", index=False)

data_test_clean_stem.to_csv("data/cdiscount_test_clean_stem.csv", index=False)
data_train_clean_stem.to_csv("data/cdiscount_train_clean_stem.csv", index=False)

# <FONT COLOR="Red">Épisode 2</font>

## 4 Construction des caractéristiques ou *features* par vectorisation

Les données textuelles ne peuvent pas être utilisées directment dans les différents algorithmes de d'apprentissage statistique.Plusieurs technique dites de vectorisation permettent de les traduire sous formes de vecteur numérique. Elles sont disponibles dans `Scikit-learn`:
* `One-Hot-Encoder`
* `Hashing`
* `Tf-Idf`

### 4.1 Lecture des données netoyées des étapes précédentes

In [ ]:
data_test_clean_stem = pd.read_csv("data/cdiscount_test_clean_stem.csv").fillna("")
data_train_clean_stem = pd.read_csv("data/cdiscount_train_clean_stem.csv").fillna("")

Le dossier `features` créé ci-dessous contiendra les *data frames* des différentes possibilités de vectorisation.

In [ ]:
import os
DATA_OUTPUT_DIR = "data/features"
if not(os.path.isdir("data/features")):
    os.mkdir("data/features")

Seule la colonne *Description* des `data frames` est considérée. 

In [ ]:
train_array = data_train_clean_stem["Description"].values
test_array = data_test_clean_stem["Description"].values

### 4.2 Vectorisation par  *One-Hot-Encoding*
Les mots sont codés par des présences / absences. Il est également possible de prendre en compte des n-grammes. Les matrices ainsi crées sont stockés dans un format creux ou parcimonieux (*sparse*).

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

extr_cv = CountVectorizer(binary=False)
data_train_OHE = extr_cv.fit_transform(train_array)
data_train_OHE

In [ ]:
vocabulary = extr_cv.get_feature_names()
N_vocabulary = len(vocabulary)
N_vocabulary

Affiche la première ligne

In [ ]:
ir = 0

rw = data_train_OHE.getrow(ir)
print(train_array[ir])
pd.DataFrame([(v, vocabulary[v], k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","token","weight"])


In [ ]:
data_test_OHE = extr_cv.transform(test_array)
data_test_OHE

In [ ]:
ir = 5

rw = data_test_OHE.getrow(ir)
print(test_array[ir])
pd.DataFrame([(v, vocabulary[v], k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","token","weight"])


### 4.3 Vectorisation par TF-IDF¶

Le *TF-IDF* permet de faire ressortir l'importance relative de chaque mot $m$, ou couples de mots consécutifs (bi-gram*) dans un texte-produit ou un descriptif $d$, par rapport à la liste entière des produits ou descriptifs. La fonction $TF(m,d)$ compte le nombre d'occurences du mot $m$ dans le descriptif $d$. La fonction $IDF(m)$ mesure l'importance du terme dans l'ensemble des documents ou descriptifs en donnant plus de poids aux termes les moins fréquents car considérés comme les plus discriminants (motivation analogue à celle de la métrique du chi2 en analyse des correspondance). $IDF(m,l)=\log\frac{D}{f(m)}$ où $D$ est le nombre de documents, la taille de l'échantillon d'apprentissage, et $f(m)$ le nombre de documents ou descriptifs contenant le mot $m$. La nouvelle variable ou *features* est $V_m(l)=TF(m,l)\times IDF(m,l)$.

Comme pour les transformations des variables quantitatives (centrage, réduction), la même transformation c'est-à-dire les mêmes pondérations, est calculée sur l'achantillon d'apprentissage et appliquée à celui de test. La fonction `TfidfVectorizer` opèrecette transformation en produisant une matrice creuse. Fixer le paramètre `norm = False` rend les résultats plus explicite.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer( ngram_range=(1,1), norm = False)
data_train_TFIDF = vec.fit_transform(train_array)
data_train_TFIDF

In [ ]:
vocabulary = vec.get_feature_names()
N_vocabulary = len(vocabulary)
N_vocabulary

In [ ]:
ir = 0

rw = data_train_TFIDF.getrow(ir)
print(train_array[ir])
pd.DataFrame([(v, vocabulary[v], vec.idf_[v], k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","token","idf","weight"])


La fonction `TfidfVectorizer` admet plusieurs variantes: *smooth idf, sublinear_tf* ainsi que la possibilité de prendre en compte des n-gram mais la taille du dictionnaire explose.

La même transformation de vectorisation est ensuite appliquée sur le jeu de données test.

In [ ]:
data_test_TFIDF = vec.transform(test_array)
data_test_TFIDF

Le *tf* est évalué pour chaque ligne ou descriptif mais pondéré par le même *idf*.

In [ ]:
ir = 5

rw = data_test_TFIDF.getrow(ir)
print(test_array[ir])
pd.DataFrame([(v, vocabulary[v], vec.idf_[v], k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","token","idf","weight"])


### 4.4 Fonction de Hashage

Le *hashnig* permet de réduire l'espace des variables (taille du dictionnaire) en un nombre limité et fixé a priori `n_hash` de caractéristiques ou *features*. Il repose sur la définition d'une fonction de hashage $h$ qui à un indice $j$ défini dans l'espace des entiers naturels, renvoie de façon déterministe et injective un indice $i=h(j)$ dans dans l'espace réduit (1 à n_hash) des caractéristiques. Ainsi le poids de l'indice $i$, du nouvel espace, est l'association de tous les poids d'indice $j$ tels que $i=h(j)$ de l'espace originale. Ici, les poids sont associés d'après la méthode décrite par Weinberger et al. (2009).

N.B. $h$ n'est pas généré aléatoirement. Ainsi pour un même fichier d'apprentissage (ou de test) et pour un même entier `n_hash`, le résultat de la fonction de hashage est identique

Le dictionnaire est pris en paramètre d'entrée.

In [ ]:
from sklearn.feature_extraction import FeatureHasher
nb_hash = 300

feathash = FeatureHasher(nb_hash)
train_dict_array  = map(lambda x : collections.Counter(x.split(" ")), train_array)
data_train_hash = feathash.fit_transform(train_dict_array)

In [ ]:
data_train_hash

In [ ]:
ir = 0

rw = data_train_hash.getrow(ir)
print(train_array[ir])
pd.DataFrame([(v, k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","weight"])


Cette fonction réduit la taille de la matrice, mais il n'existe pas de fonction inverse explicite à cette transformation. Le résultat perd en lisibilit ou interprétabilité. C'est néanmoins indispensbale si le dictionnaire est très volumineux. Pour la partie d'apprentissage, il est possible d'utiliser la matrice de hashage obtenue comme des présence absence ou encore de lui associer les pondérations issues d'une transformation TF-IDF comme ci-dessous.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

vec =  TfidfTransformer(norm = False)
data_train_HTfidf = vec.fit_transform(data_train_hash)
data_train_HTfidf

In [ ]:
ir = 0

rw = data_train_HTfidf.getrow(ir)
print(train_array[ir])
pd.DataFrame([(v, vec.idf_[v], k)  for k,v in zip(rw.data, rw.indices)], columns=["indices","idf_","weight"])


### 4.5 Fonctions de Vectorisation

Ces transformaitons asdmettent de nombreuses options dont il faut fixer les valeurs des paramètres. Dans les présents tests, seuls 4 jeux d'apprentissage et test sont construits:
* `count` avec des occurences (*one hot*) de 0,1 
   - sans hashage 
   - avec `n_hash=300`
* `TFIDF` 
   - sans hashage 
   - avec `n_hash=300`

D'autres options pourraient être considérées en introduisant des n-grams ou en faisant varier `n_hash` ou encore les variantes de *TFIDF*. 

Les deux fonctions `vectorizer_train` and `apply_vectorizer` permettent de générer les différentes options sur l'échantillon d'apprentissage puis de les appliquéer également à l'échantillon test.

In [ ]:
def vectorizer_train(df, columns=['Description'], nb_hash=None, nb_gram = 1, vectorizer = "tfidf" , binary = False):
    
    data_array = [" ".join(line) for line in df[columns].values]
    
    # Hashage
    if nb_hash is None:
        feathash = None
        if vectorizer == "tfidf":
            vec = TfidfVectorizer(ngram_range=(1,nb_gram))
            data_vec = vec.fit_transform(data_array)
        else:
            vec = CountVectorizer(binary=binary)
            data_vec = vec.fit_transform(data_array)
    else:
        data_dic_array = [collections.Counter(line.split(" ")) for line in data_array]
        feathash = FeatureHasher(nb_hash)
        data_hash = feathash.fit_transform(data_dic_array)
        
        if vectorizer=="tfidf":
            vec =  TfidfTransformer()
            data_vec =  vec.fit_transform(data_hash)
        else:
            vec = None
            data_vec = data_hash

    return vec, feathash, data_vec



def apply_vectorizer(df, vec, feathash, columns =['Description', 'Libelle', 'Marque']):
    
    data_array = [" ".join(line) for line in df[columns].values]
    
    #Hashage
    if feathash is None:
        data_hash = data_array
    else:
        data_dic_array = [collections.Counter(line.split(" ")) for line in data_array]
        data_hash = feathash.transform(data_dic_array)
    
    if vec is None:
        data_vec = data_hash
    else:
        data_vec = vec.transform(data_hash)
    return data_vec


Création des couples de données apprentissage x test.

In [ ]:
parameters = [[None, "count"],
              [300, "count"],
              [10000,"count"],
              [None, "tfidf"],
              [300, "tfidf"],
             [10000,"tfidf"]]

from scipy import sparse

for nb_hash, vectorizer in parameters:
    ts = time.time()
    vec, feathash, data_train_vec = vectorizer_train(data_train_clean_stem, nb_hash=nb_hash, vectorizer = vectorizer)
    data_test_vec = apply_vectorizer(data_test_clean_stem, vec, feathash)
    te = time.time()
    
    print("nb_hash : " + str(nb_hash) + ", vectorizer : " + str(vectorizer))
    print("Runing time for vectorization : %.1f seconds" %(te-ts))
    print("Train shape : " + str(data_train_vec.shape))
    print("Test shape : " + str(data_test_vec.shape))

    
    sparse.save_npz(DATA_OUTPUT_DIR +"/vec_train_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer), data_train_vec)
    sparse.save_npz(DATA_OUTPUT_DIR +"/vec_test_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer), data_test_vec)
    

# <FONT COLOR="Red">Épisode 3</font>

## 5 Prévision de la catégorie d'un texte.
Les deux vectorisations sont utilisées avec différentes valeurs de `n_hash` pour entraîner un algorithme d'apprentissage afin de prévoir les catégories de l'échantillon test puis comparer les erreurs de prévision. 

Deux algorihtmes sont utilisés: la régression logistique et les forêts aléatoires.

**Attention** les temps d'exécution sont assez long...

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

Y_train = pd.read_csv("data/cdiscount_train_subset.csv").fillna("")["Categorie1"]
Y_test = pd.read_csv("data/cdiscount_test.csv").fillna("")["Categorie1"]

### 5.1 Régression logistique

In [ ]:
DATA_DIR = "data/features"
from scipy import sparse
metadata_list_lr = []
parameters = [[None, "count"],
              [300, "count"],
              [10000, "count"],
              [None, "tfidf"],
              [300, "tfidf"],
              [10000, "tfidf"],]

for nb_hash, vectorizer in parameters:
    print("nb_hash : " + str(nb_hash) + ", vectorizer : " + str(vectorizer))
    X_train = sparse.load_npz(DATA_DIR +"/vec_train_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer)+".npz")
    X_test = sparse.load_npz(DATA_DIR +"/vec_test_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer)+".npz")
    ts = time.time()
    cla = LogisticRegression(n_jobs=-1)
    cla.fit(X_train,Y_train.values)
    te=time.time()
    t_learning = te-ts
    ts = time.time()
    score_train=cla.score(X_train,Y_train)
    score_test=cla.score(X_test,Y_test)
    te=time.time()
    t_predict = te-ts
    metadata = {"typeW2V": None, "nb_hash": nb_hash, "vectorizer":vectorizer , "learning_time" : t_learning, "predict_time":t_predict, "score_train": score_train, "score_test": score_test}
    print(metadata)
    metadata_list_lr.append(metadata)
       

### 5.2 Forêt aléatoire

In [ ]:
from sklearn.ensemble import RandomForestClassifier
metadata_list_rf = []

parameters = [[None, "count"],
              [300, "count"],
              [10000, "count"],
              [None, "tfidf"],
              [300, "tfidf"],
              [10000, "tfidf"],]

for nb_hash, vectorizer in parameters:
    print("nb_hash : " + str(nb_hash) + ", vectorizer : " + str(vectorizer))
    X_train = sparse.load_npz(DATA_DIR +"/vec_train_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer)+".npz")
    X_test = sparse.load_npz(DATA_DIR +"/vec_test_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer)+".npz")
    ts = time.time()
    cla = RandomForestClassifier(n_estimators=100,n_jobs=-1)
    cla.fit(X_train,Y_train.values)
    te=time.time()
    t_learning = te-ts
    ts = time.time()
    score_train=cla.score(X_train,Y_train)
    score_test=cla.score(X_test,Y_test)
    te=time.time()
    t_predict = te-ts
    metadata = {"typeW2V": None, "nb_hash": nb_hash, "vectorizer":vectorizer , "learning_time" : t_learning, "predict_time":t_predict, "score_train": score_train, "score_test": score_test}
    print(metadata)
    metadata_list_rf.append(metadata)

#### Comparaisons

In [ ]:
pd.DataFrame(metadata_list_lr)

In [ ]:
pd.DataFrame(metadata_list_rf)

#### Conclusion?

# <FONT COLOR="Red">Épisode 4</font>

## 6 Vectorisation par  `Word2Vec`
Cette fonction consiste à opérer un *plongement* (*embedding*) du dictionnaire dans un espace vectoriel de dimension fixé. Chaque mot est alors un vecteur de cet espace dont grosso-modo la proximité au sens de la métrique euclidienne est fonction des co-occurences des mots dans un proche voisinage au sein d'un même texte. Le voisinnage est défini par une fenêtre de taille ou nombre de mots consécutifs fixé. 

Cette fonction est techniquement l'application d'un algorithme *autoencoder-decoder* à trois couches. La couche d'entrée est celle de sortie ont la taille du dictionnaire, celle cachée a pour taille la dimension de l'espace vectoriel. Deux options sont disponibles `CBOW` ( continuous bag-of-words) ou `skip-gram` selon que le mot est entrée associé en sortie avec les mots présents dans la fenètre ou *vice-versa*. 

La fonction `word2vec` intègre à sa façon l'environnement de chaque mot et donc une part séantique de chaque texte mais san sprende en compte l'ordre des mots.

De très nombreuses astuces algorithmiques permettent d'évaluer cette matrice de vecteurs (un par mot) même sur des corpus de donées très volumineux comme les *google news* ou les pages Wikipedia dans une langue déterminée. 

Il est ainsi possible d'apprendre un "modèle" à partir de son corpus de données ou encore de charger un module déjà appris. [Kyubyong Park](https://github.com/Kyubyong/wordvectors) propose des modèles pour une trentaine de langues dont le français.

La librairie `gensim` opère ce *Word Embedding* par `Word2Vec`.

In [ ]:
import gensim

import nltk 
stemmer=nltk.stem.SnowballStemmer('french')

### 6.1 Construire un modèle  Word2Vec
Préparation des données et choix des options.

In [ ]:
train_array_token = [line.split(" ") for line in train_array]
test_array_token = [line.split(" ") for line in test_array]

Choix des paramètres ci-dessous:

La valeur de `sg` détermine l'option: "1" pour `skip-gram` ou "0" pour `CBOW`. 

`hs` dirige les options de l'algorihtme. 
* `hs=1` l'entraînement est opéré avec une fonction de `softmax` hiérarchique
* `hs=0` et `negative` entier entre 5 et 20. Spécifie combien de "nots bruits" sont tirés aléatoirement parmi ceux qui ne sont pas dans la fenêtre. 

In [ ]:
Features_dimension = 300
sg = 1
hs = 0
negative = 10
X = train_array_token
N_train = len(X)

Estimation des modèles à partir du corpus de textes pour les deux modes de l'algorihtme: `skip-gram`  ou `CBOW`.

In [ ]:
model_dic = {}
for sg in [0,1]:
    print("Start learning Word2Vec learning")
    print("Params are : Fdim_%d_sg_%d_hs_%d_negative_%d_model" %(Features_dimension, sg, hs, negative))
    ts = time.time()
    model = gensim.models.Word2Vec(X, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
    te = time.time()
    t_learning = te-ts

    # Metadata
    N_vocab, feature_dim = model.wv.vectors.shape
    metadata = {"learning_time" : t_learning, "vocab_size" : N_vocab, "sg" : sg, "negative": negative, "hs":hs}
    print(metadata)
    
    model_name = "skip-gram" if sg==1 else "CBOW"
    model_dic.update({model_name : model})

Charger puis décompresser le modèle de  [Kyubyong Park](https://github.com/Kyubyong/wordvectors) pour le français. dans le répertoire `data` avant de charger ce modèle `on line`.

In [ ]:
model_online_dir = "data/fr/fr.bin"
model = gensim.models.Word2Vec.load(model_online_dir)
model_dic.update({"online" : model})

### 6.2 Utilisations de Word2Vec en NLP
Cette section illustre quelques unes des fonctions classiques d'un modèle Word2Vec avant de revenir à l'objectif de classificaiton supervisée.
#### Mots les plus similaires
Les résultats sont pertinent avec un volume suffisant; *e.g.* `N_train=1.000.000`; l'effectif de 100.000 est trop réduit.

In [ ]:
model.predict_output_word(["homm"])

In [ ]:
term="homme"

df_ = []
columns = []
for model_name, model in model_dic.items():
    token = stemmer.stem(term) if "online"!=model_name else term
    mpow = model.wv.most_similar([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

In [ ]:
term="femme"

df_ = []
columns = []
for model_name, model in model_dic.items():
    token = stemmer.stem(term) if "online"!=model_name else term
    mpow = model.wv.most_similar([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

In [ ]:
term="xbox"

df_ = []
columns = []
for model_name, model in model_dic.items():
    token = stemmer.stem(term) if "online"!=model_name else term
    mpow = model.wv.most_similar([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

#### Combinaison de mots

In [ ]:
terms_positif = ["femme","roi"]
terms_negatif = ["homme"]

df_ = []
columns = []
for model_name, model in model_dic.items():
    token_positif = [stemmer.stem(term) if "online"!=model_name else term for term in terms_positif]
    token_negativ = [stemmer.stem(term) if "online"!=model_name else term for term in terms_negatif]
    
    mpow = model.wv.most_similar(positive=token_positif, negative=token_negativ)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

In [ ]:
terms_positif = ["espagne","paris"]
terms_negatif = ["france"]

df_ = []
columns = []
for model_name, model in model_dic.items():
    token_positif = [stemmer.stem(term) if "online"!=model_name else term for term in terms_positif]
    token_negativ = [stemmer.stem(term) if "online"!=model_name else term for term in terms_negatif]
    
    mpow = model.wv.most_similar(positive=token_positif, negative=token_negativ)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

#### Compléter une séquence

In [ ]:
terms = ["voir","la"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

In [ ]:
terms = ["coque",'pour',"samsung"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

In [ ]:
terms = ["homme"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

In [ ]:
terms = ["femme"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

### 6.3 Construire des *features* avec *Word2Vec*
Un texte ou descriptif d'article est représenté par le barycentre des mots qui le composent. Ce barycentre peut éventuellement être calculé avec des pondérations. Le nombre de caractéristiques ou variables décrivant chaque texte est la dimension de l'espace des mots, ici 300. C'est donc une façon de prendre en compte le contexte des mots tout en réduisant drastiquement la dimension de l'espace.

#### Avec les modèles appris
Prise ne compte des textes puis définition des fonctions.

In [ ]:
data_test_clean = pd.read_csv("data/cdiscount_test_clean.csv").fillna("")
data_train_clean = pd.read_csv("data/cdiscount_train_clean.csv").fillna("")

train_array_token_wstem = [line.split(" ") for line in data_train_clean["Description"].values]
test_array_token_wstem = [line.split(" ") for line in data_test_clean["Description"].values]

In [ ]:
def get_features_mean(lines):
    features = [MODEL[x] for x  in lines if x in MODEL]
    if features == []:   
        fm =np.ones(F_SIZE)
    else :
        fm = np.mean(features,axis=0)
    return fm

def get_matrix_features_means(X):
    X_embedded_ = list(map(get_features_mean, X))
    X_embedded = np.vstack(X_embedded_)
    return X_embedded

Calcul de la matrice des caractéristiques ou *features* pour les trois modèles.

In [ ]:
for model_name in ["CBOW","skip-gram", "online"]:
    
    if "online" == model_name:
        X_train = train_array_token_wstem
        X_test = test_array_token_wstem
    else:
        X_train = train_array_token
        X_test = test_array_token
    
    model = model_dic[model_name]
    MODEL = model
    F_SIZE = Features_dimension

    ts = time.time()
    X_embedded_train = get_matrix_features_means(X_train)
    te = time.time()
    t_train = te-ts
    #np.save(embedded_train_dir, X_embedded_train)
    print("Time conversion : %d seconds"%t_train)
    print("Shape Matrix : (%d,%d)"%X_embedded_train.shape)
    np.save(DATA_OUTPUT_DIR +"/embedded_train_nb_hash_"+model_name, X_embedded_train)

    ts = time.time()
    X_embedded_test = get_matrix_features_means(X_test)
    te = time.time()
    t_test = te-ts
    #np.save(embedded_test_dir, X_embedded_test)
    print("Time conversion : %d seconds"%t_test)
    print("Shape Matrix : (%d,%d)"%X_embedded_test.shape)
    np.save(DATA_OUTPUT_DIR +"/embedded_test_nb_hash_"+model_name, X_embedded_test)
    
    metadata = {"t_train" : t_train, "t_test" : t_test, "sg":sg}
    print(metadata)

In [ ]:
model["homme"]

In [ ]:
model.wv.syn0[model.wv.vocab["homme"].index] == model["homme"]

In [ ]:
model.syn1neg.shape

In [ ]:
vh = model.wv.vocab["homme"]
vh.index

# <FONT COLOR="Red">Épisode 5</font>

## 7 Prévision de la catégorie d'un texte.
La vectorisation issue de Word2Vec est utilisée pour entraîner un algorithme d'apprentissage afin de prévoir les catégories de l'échantillon test et mesurer puis comparer les erreurs de prévision obtenues à la suite des vectorisation précédentes.

Deux algorihtmes sont utilisés: la régression logistique et les forêts aléatoires.

**Attention** les temps d'exécution sont assez long...

Fixer le paramètre `njobs=-1` permet d'activer tous les processeurs. Le gain de temps est très sensible pour les forêts aléatoires mais cette valeur n'est pas permise pour le solveur utilisé par défaut en réression logistique. Il faudrait sans doute changer cette dernière option.

In [ ]:
Y_train = pd.read_csv("data/cdiscount_train_subset.csv").fillna("")["Categorie1"]
Y_test = pd.read_csv("data/cdiscount_test.csv").fillna("")["Categorie1"]

### 7.1 Régression Logistique

In [ ]:
for model_name in ["CBOW","skip-gram", "online"]:
    print("Word2Vec :" + model_name)

    X_train = np.load(DATA_DIR +"/embedded_train_nb_hash_" + model_name+".npy")
    X_test = np.load(DATA_DIR +"/embedded_test_nb_hash_" + model_name+".npy")
    
    ts = time.time()
    cla = LogisticRegression()
    cla.fit(X_train,Y_train.values)
    te=time.time()
    t_learning = te-ts
    ts = time.time()
    score_train=cla.score(X_train,Y_train)
    score_test=cla.score(X_test,Y_test)
    te=time.time()
    t_predict = te-ts
    metadata = {"typeW2V": model_name ,"nb_hash": None, "vectorizer":"word2vec" ,"learning_time" : t_learning, "predict_time":t_predict, "score_train": score_train, "score_test": score_test}
    print(metadata)
    metadata_list_lr.append(metadata)


In [ ]:
np.load("data/features/embedded_train_nb_hash_CBOW.npy").shape

#### Comparaison des résultats

In [ ]:
pd.DataFrame(metadata_list_lr)

### 6.2 Forêt aléatoire

In [ ]:
for model_name in ["CBOW","skip-gram", "online"]:
    print("Word2Vec :" + model_name)

    X_train = np.load(DATA_DIR +"/embedded_train_nb_hash_" + model_name+".npy")
    X_test = np.load(DATA_DIR +"/embedded_test_nb_hash_" + model_name+".npy")
    
    ts = time.time()
    cla = RandomForestClassifier(n_estimators=100,n_jobs=-1)
    cla.fit(X_train,Y_train.values)
    te=time.time()
    t_learning = te-ts
    ts = time.time()
    score_train=cla.score(X_train,Y_train)
    score_test=cla.score(X_test,Y_test)
    te=time.time()
    t_predict = te-ts
    metadata = {"typeW2V": model_name ,"nb_hash": None, "vectorizer":"word2vec" ,"learning_time" : t_learning, "predict_time":t_predict, "score_train": score_train, "score_test": score_test}
    print(metadata)
    metadata_list_rf.append(metadata)


#### Comparaison des résultats

In [ ]:
pd.DataFrame(metadata_list_lr)

#### Conclusion ?